In [1]:
# Import PySpark
from pyspark.sql import SparkSession

#Create SparkSession
spark = SparkSession.builder.appName('SparkCleaning').getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/09 15:55:34 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
spark

In [3]:
df_spark = spark.read.csv("../datasets/imdb/validation_hidden.csv", inferSchema=True, header=True)
df_spark.repartition(4)

DataFrame[_c0: int, tconst: string, primaryTitle: string, originalTitle: string, startYear: string, endYear: string, runtimeMinutes: string, numVotes: double]

In [4]:
df_spark.show()

23/03/09 15:55:40 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , tconst, primaryTitle, originalTitle, startYear, endYear, runtimeMinutes, numVotes
 Schema: _c0, tconst, primaryTitle, originalTitle, startYear, endYear, runtimeMinutes, numVotes
Expected: _c0 but found: 
CSV file: file:///Users/mattiacintioli/PycharmProjects/Big-Data-Project/datasets/imdb/validation_hidden.csv
+---+---------+--------------------+-------------------+---------+-------+--------------+--------+
|_c0|   tconst|        primaryTitle|      originalTitle|startYear|endYear|runtimeMinutes|numVotes|
+---+---------+--------------------+-------------------+---------+-------+--------------+--------+
|  0|tt0003740|             Cabiria|               null|     1914|     \N|           148|  3452.0|
|  1|tt0008663|     A Man There Was|        Terje Vigen|     1917|     \N|            65|  1882.0|
|  3|tt0010307|           J'accuse!|               null|     1919|     \N|           166|  1692.0|

In [5]:
df_spark.count()

955

Starting to do stuff

In [6]:
import pandas as pd
import os

In [7]:
import os

dir_path = "../datasets/imdb/"

# Get a list of all the files in the directory
files = os.listdir(dir_path)

# Filter the list to include only files that start with "train-" and have a ".csv" extension
train_files = [f for f in files if f.startswith("train-") and f.endswith(".csv")]

# Count the number of files
num_train_files = len(train_files)

print("Number of train files: {}".format(num_train_files))


Number of train files: 8


In [8]:
# Create an empty list to store the DataFrames
dfs = []

# Loop through the file names and read each file into a DataFrame
for file in train_files:
    file_name = f"../datasets/imdb/{file}"
    df = spark.read.csv(file_name, header=True, inferSchema=True)
    dfs.append(df)

# Loop through the DataFrames and print the shape of each one
for i, df in enumerate(dfs):
    num_rows = df.count()
    num_cols = len(df.columns)
    print("Shape of df_train_{}: {} rows, {} columns".format(i+1, num_rows, num_cols))

# Union all the DataFrames into one big DataFrame
big_df = dfs[0]
for df in dfs[1:]:
    big_df = big_df.union(df)
num_rows = big_df.count()
num_cols = len(big_df.columns)
print(f"\nShape of big_df: {num_rows} rows, {num_cols} columns")

Shape of df_train_1: 1001 rows, 9 columns
Shape of df_train_2: 1035 rows, 9 columns
Shape of df_train_3: 974 rows, 9 columns
Shape of df_train_4: 1049 rows, 9 columns
Shape of df_train_5: 963 rows, 9 columns
Shape of df_train_6: 951 rows, 9 columns
Shape of df_train_7: 993 rows, 9 columns
Shape of df_train_8: 993 rows, 9 columns

Shape of big_df: 7959 rows, 9 columns


In [9]:
big_df.show()

23/03/09 15:55:43 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , tconst, primaryTitle, originalTitle, startYear, endYear, runtimeMinutes, numVotes, label
 Schema: _c0, tconst, primaryTitle, originalTitle, startYear, endYear, runtimeMinutes, numVotes, label
Expected: _c0 but found: 
CSV file: file:///Users/mattiacintioli/PycharmProjects/Big-Data-Project/datasets/imdb/train-6.csv
+---+---------+--------------------+--------------------+---------+-------+--------------+--------+-----+
|_c0|   tconst|        primaryTitle|       originalTitle|startYear|endYear|runtimeMinutes|numVotes|label|
+---+---------+--------------------+--------------------+---------+-------+--------------+--------+-----+
| 11|tt0013257|               Häxan|               Häxan|     1922|     \N|            91| 13679.0| true|
| 12|tt0013556|          Robin Hood|                null|     1922|     \N|           143|  2178.0| true|
| 16|tt0014341|     Our Hospitality|     Our Hospitality|  

+---------+---------+
|    movie|   writer|
+---------+---------+
|tt0003740|nm0195339|
|tt0003740|nm0515385|
|tt0003740|nm0665163|
|tt0003740|nm0758215|
|tt0008663|nm0406585|
|tt0008663|nm0596410|
|tt0008663|nm0803705|
|tt0009369|nm0215874|
|tt0009369|nm0370271|
|tt0010307|nm0304098|
|tt0010600|nm0006782|
|tt0010600|nm0473134|
|tt0010600|nm0523932|
|tt0010600|nm0932559|
|tt0011439|nm0191547|
|tt0011439|nm0001196|
|tt0011439|nm1491939|
|tt0011607|nm0003433|
|tt0011607|nm0418114|
|tt0011841|nm0662461|
+---------+---------+
only showing top 20 rows

